In [ ]:
import torch
from multistage_classifier import *

In [ ]:
num_gpus = torch.cuda.device_count()
device = get_device()
device, num_gpus

In [ ]:
new_groups = (("proestrus", "estrus", "metestrus"), ("diestrus", ))
GROUP1, GROUP2 = new_groups

In [ ]:
four_dataset, _ = get_subset_dataset_and_loader("data/4_class_11", "val", batch_size=1)
# four_dataset.samples

In [ ]:
binary_dataset = regroup_dataset_classes(four_dataset, (GROUP1, GROUP2))

binary_dataloader = torch.utils.data.DataLoader(binary_dataset,
                                                batch_size=1, shuffle=True,
                                                num_workers=num_gpus * 4)

In [ ]:
binary_dataset.samples

In [ ]:
base = "models/random_affine_augment/"
model = "/resnet_transfer-101-finetune/model.pth"
model_paths = (base + "die_vs_all" + model, base + "3_class" + model)
model_paths

In [ ]:
binary_model_path = model_paths[0]

binary_meta_dict = get_meta_dict_from_model_path(binary_model_path)

binary_model = load_model(binary_model_path, binary_meta_dict, 2, device)

In [ ]:
trinary_model_path = model_paths[1]
trinary_meta_dict = get_meta_dict_from_model_path(trinary_model_path)
trinary_model = load_model(trinary_model_path, trinary_meta_dict, 3, device)

# Load Input

In [ ]:
# for all the inputs, make a prediction
input_tensor, label, path_tuple = next(iter(binary_dataloader))

input_tensor = input_tensor.to(device)
label = label.to(device)

In [ ]:
input_tensor.shape

In [ ]:
label

In [ ]:
# make prediction by passing forward through the model
with torch.set_grad_enabled(False):
    binary_raw_output = binary_model(input_tensor)
    binary_output = torch.nn.Softmax(dim=1)(binary_raw_output)

_, prediction = torch.max(binary_output, 1)

prediction

In [ ]:
# if prediction is the group of pro est met, do the 3 class classifier
if prediction == 1:
    class_names = binary_dataset.classes
    
else:
    class_names = GROUP1
    with torch.set_grad_enabled(False):
        trinary_output = torch.nn.Softmax(dim=1)(trinary_model(input_tensor))
        _, prediction = torch.max(trinary_output, 1)

prediction

In [ ]:
image_name = os.path.basename(path_tuple[0])
# get the actual string names of the class
label_class = class_names[label]
predicted_class = class_names[prediction]

image_name, label_class, predicted_class

In [ ]:
binary_output, binary_output.shape

In [ ]:
trinary_output, trinary_output.shape

In [ ]:
torch.cat((torch.t(binary_output), torch.t(trinary_output)))

In [ ]:
torch.zeros(1, 3)